# Web Scraping con Beautifulsoup y Parse
### Comprensión de estructura de datos en páginas web (fines educativos para I+D+i)

**Ariel Lujan Giamportone**

UpgradeHub - 2023

A modo de autocomprensión de aprendizaje de la implementación del Web Scraping para el análisis de datos vamos a tomar muestras de una página web. La afision por las estrellas de cine y la pantalla grande es un factor común en la sociedad. Cuando alguien quiere saber más de un actor, sus peliculas principales, conocer la trama o con quien trabajó alguno de nuestros actores preferidos se suele recurrir a los datos que proporciona IMBD (https://www.imdb.com/).

Para ejemplificar el uso del Web Scraping vamos extraer una lista de los actores de España.


## Materiales y métodos

Importar las librerias necesarias para hacer web scraping y manejo de datos:

In [31]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from parse import parse

Por conveniencia a la hora de navegar y auditar la estructura de las páginas web haremos uso del navegador Firefox Dev. Edition, pero se puede usar cualquier otro.
Hacemos la secuencia de petición (mediante la librería requests) de la página de la que queremos obtener datos:


In [32]:
headers = {'User-Agent': 'Firefox'}
url = 'https://www.imdb.com/search/name/?birth_place=spain&adult=include&count=100'
pagina = requests.get(url,headers=headers)


Vamos a utilizar la libreria BeautifulSoup para analizar HTML o XML, proporcionando modismos o expresiones propias de Python para iterar, buscar y modificar el árbol.

In [33]:
sopa = BeautifulSoup(pagina.text, 'html')
sopa

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1703286451884);
        }
    })</script><title>Advanced search</title><meta content="" data-id="main" name="description"/><meta content="https://www.imdb.com/search/" property="og:url"/><meta content="IMDb" property="og:site_name"/><meta content="Advanced search" property="og:title"/><meta content="" property=

Mediante el inspector del navegador podemos analizar la estructura del código HTML y posicionarnos sobre lo que nos interesa extraer para nuestra tabla de datos. La regla típica de jerarquía basada en HTML es recorrer la sintaxis en el siguiente orden:
1º ID
2º Class name
3º Tag Name, CSS selector
4º Xpath 

In [34]:
actores = sopa.find_all('li', class_ = 'ipc-metadata-list-summary-item')
print(type(actores))
print(len(actores))

<class 'bs4.element.ResultSet'>
100


In [35]:
primer_actor = actores[0]
print(primer_actor.prettify())

<li class="ipc-metadata-list-summary-item">
 <div class="ipc-metadata-list-summary-item__c">
  <div class="ipc-metadata-list-summary-item__tc">
   <span aria-disabled="false" class="ipc-metadata-list-summary-item__t">
   </span>
   <div class="sc-81130baa-0 Mrwos dli-parent">
    <div class="sc-d4d25ac8-0 gDeuyD">
     <div class="sc-d4d25ac8-1 eOHOAY" data-testid="nlib-img-container">
      <div class="ipc-avatar ipc-avatar--base" role="group">
       <div class="ipc-media ipc-media--avatar ipc-image-media-ratio--avatar ipc-media--base ipc-media--custom ipc-media--avatar-circle ipc-avatar__avatar-image ipc-media__img" style="width:60px">
        <img alt="Millie Bobby Brown at an event for The 70th Primetime Emmy Awards (2018)" class="ipc-image" loading="lazy" sizes="50vw, (min-width: 480px) 34vw, (min-width: 600px) 26vw, (min-width: 1024px) 16vw, (min-width: 1280px) 16vw" src="https://m.media-amazon.com/images/M/MV5BMjA5NzA0NzQzMF5BMl5BanBnXkFtZTgwMTQxNjUzNjM@._V1_QL75_UX60_CR0,0,60,

Nos posicionamos en lo que deseamos extraer:

In [36]:
primer_actor.h3.text.split('. ')[-1]

'Millie Bobby Brown'

Extraemos la imágen de los actores:

In [37]:
actores[1].img['src']

'https://m.media-amazon.com/images/M/MV5BMzk1YTFkNmItZGUzOC00YzVlLTkwOGUtMDY0YTAyYmEwYWZlXkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_QL75_UX60_CR0,0,60,60_.jpg'

Extraemos la biografía deseada:

In [38]:
primer_actor.find('div',class_='ipc-html-content-inner-div').text

'Millie Bobby Brown (born 19 February 2004) is an English actress and model. She rose to prominence for her role as Eleven in the Netflix science fiction drama series Stranger Things (2016), for which she earned a Primetime Emmy Award nomination for Outstanding Supporting Actress in a Drama Series at age 13. She is also the youngest person ever to feature on TIME 100 list.Millie was born in Marbella, Andalusia, Spain, the third of four children of English parents, Kelly and Robert Brown. The family moved to Bournemouth, Dorset, when Brown was around four years old, and then to Orlando, Florida, four years later. Here, Millie went to acting workshops to pass time on a Saturday, and it was there that a top Hollywood talent scout called and told Millie\'s parents that "she has instincts you cannot teach." She advised Millie\'s parents that she could "mix it with the best kids in Hollywood." They packed up and drove from Orlando to Los Angeles, and within a week, Millie was meeting with th

Generamos un bucle para automatizar la extracción de cada uno respetando la estructura de la web:

In [39]:
nombres = []
imagenes = []
bios = []

actores = sopa.find_all('li', class_ = 'ipc-metadata-list-summary-item')

for actor in actores:
    
    nombre = actor.h3.text.split('. ')[-1]
    nombres.append(nombre)
    
    if actor.img == None:
        imagen='https://i.pinimg.com/736x/1a/c9/23/1ac923692e2a58fd5fbc9766ae519ebf.jpg'
    else :
        imagen = actor.img['src']
    imagenes.append(imagen)
    
    bio = actor.find('div',class_='ipc-html-content-inner-div').text
    bios.append(bio)

Convertimos los datos a un Data Frame con Pandas para poder trabajar con ellos en el preprocesamiento y análisis.

In [40]:
import pandas as pd
informacion_actores = pd.DataFrame({'Nombre': nombres,
'Imagen': imagenes,
'Bio': bios
})
print(informacion_actores.info())
informacion_actores.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Nombre  100 non-null    object
 1   Imagen  100 non-null    object
 2   Bio     100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB
None


Nombre  \
0  Millie Bobby Brown   
1          Dafne Keen   
2    Antonio Banderas   
3       Javier Bardem   
4       Penélope Cruz   
5         Elsa Pataky   
6        Oona Chaplin   
7        Daniel Brühl   
8        Marta Milans   
9          Taz Skylar   

                                                                                                                                                 Imagen  \
0                                   https://m.media-amazon.com/images/M/MV5BMjA5NzA0NzQzMF5BMl5BanBnXkFtZTgwMTQxNjUzNjM@._V1_QL75_UX60_CR0,0,60,60_.jpg   
1   https://m.media-amazon.com/images/M/MV5BMzk1YTFkNmItZGUzOC00YzVlLTkwOGUtMDY0YTAyYmEwYWZlXkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_QL75_UX60_CR0,0,60,60_.jpg   
2                                   https://m.media-amazon.com/images/M/MV5BMTUyOTQ3NTYyNF5BMl5BanBnXkFtZTcwMTY2NjIzNQ@@._V1_QL75_UX60_CR0,0,60,60_.jpg   
3                                   https://m.media-amazon.com/images/M/MV5BMTY1NTc4NTYzMF5BMl5BanBnXkFtZTcwNDIwOTY1NA@@._V1_QL75_UX60_CR0,0,60,60_.jpg   
4                                   https://m.media-amazon.com/images/M/MV5BMTM0Mzc1MTc2OF5BMl5BanBnXkFtZTcwMzE4MzQxMw@@._V1_QL75_UX60_CR0,0,60,60_.jpg   
5                                   https://m.media-amazon.com/images/M/MV5BMTAyNjU4MzkyNzleQTJeQWpwZ15BbWU3MDk2NTQ0Mjk@._V1_QL75_UX60_CR0,0,60,60_.jpg   
6                                   https://m.media-amazon.com/images/M/MV5BMjkwMjAyNjY4MF5BMl5BanBnXkFtZTcwMjA2MDg0Nw@@._V1_QL75_UX60_CR0,0,60,60_.jpg   
7   https://m.media-amazon.com/images/M/MV5BMzliYzY1NWQtNTFhYy00NjVhLTlkOGItY2RhNDgwOTg5ZDhiXkEyXkFqcGdeQXVyNTkzODg0ODk@._V1_QL75_UX60_CR0,0,60,60_.jpg   
8  https://m.media-amazon.com/images/M/MV5BY2I4ZTE0NTgtOWUzOC00ZWNkLWE4NjAtMjljMTVkOTc4ODdjXkEyXkFqcGdeQXVyMTk0ODI1Mjc@._V1_QL75_UY60_CR15,0,60,60_.jpg   
9   https://m.media-amazon.com/images/M/MV5BZjNkZjU3NjMtOWQwYy00Y2RhLTk4MmMtMmYwZjFhOWU1MDE5XkEyXkFqcGdeQXVyNjI0MzA4MTQ@._V1_QL75_UX60_CR0,0,60,60_.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

Para obtener una visualización más atractiva:

In [41]:
# Para imprimir la foto, usamos el siguiente código:
from IPython.core.display import HTML

# Convertir una URL en una iamgen
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

# informacion_actores: Dataframe con la información
# Imagen: Columna con la URL de las imánes de los actores y actrices
HTML(informacion_actores.to_html(escape=False ,formatters=dict(Imagen=path_to_image_html)))